In [2]:
import pandas as pd
from datetime import datetime
import ee

# Caminho para o arquivo JSON de credenciais
caminho_credenciais = 'gee.json'

# Inicializar a autenticação usando a função ServiceAccountCredentials
credenciais = ee.ServiceAccountCredentials('geeprojeto@ee-marcosadassan.iam.gserviceaccount.com', caminho_credenciais)
ee.Initialize(credenciais)

In [ ]:
# Define o ponto de interesse
import time
ponto = ee.Geometry.Point([-50.39405389922201, -20.959639053569816])

# GPM V7 dados de 30 minutos para um único dia
dataset = ee.ImageCollection('NASA/GPM_L3/IMERG_V07') \
    .filter(ee.Filter.date('2024-11-15', '2024-12-15')) \
    .select('precipitation')


# Função para obter o valor de precipitação no ponto para cada imagem
def get_precipitation_at_point(image: ee.image.Image):
    value = image.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=ponto,
        scale=1000
    ).get('precipitation')
    
    # Converte a precipitação de mm/h para mm em 30 minutos
    precipitation_30min = ee.Number(value).divide(2)
    
    return image.set('stats', ee.Dictionary({"precipitation_30min":precipitation_30min, 'date': image.date().format('YYYY-MM-dd HH:mm:ss')}))

# Aplica a função a cada imagem na coleção
values_at_point = dataset.map(get_precipitation_at_point).aggregate_array('stats')


# Coleta os valores em uma lista para inspeção
values_list = values_at_point.getInfo()

# Dicionário para armazenar a precipitação acumulada por data
daily_accumulation = {}

# Calcula a precipitação acumulada diária
for v in values_list:
    date_str = v["date"]
    date_obj = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S').date()
    precipitation = v['precipitation_30min']

    
    daily_accumulation[date_obj] = daily_accumulation.get(date_obj, 0) + precipitation

return [{k:v} for k, v in daily_accumulation.items()]




2024-11-15 00:00:00
2024-11-15 00:30:00
2024-11-15 01:00:00
2024-11-15 01:30:00
2024-11-15 02:00:00
2024-11-15 02:30:00
2024-11-15 03:00:00
2024-11-15 03:30:00
2024-11-15 04:00:00
2024-11-15 04:30:00
2024-11-15 05:00:00
2024-11-15 05:30:00
2024-11-15 06:00:00
2024-11-15 06:30:00
2024-11-15 07:00:00
2024-11-15 07:30:00
2024-11-15 08:00:00
2024-11-15 08:30:00
2024-11-15 09:00:00
2024-11-15 09:30:00
2024-11-15 10:00:00
2024-11-15 10:30:00
2024-11-15 11:00:00
2024-11-15 11:30:00
2024-11-15 12:00:00
2024-11-15 12:30:00
2024-11-15 13:00:00
2024-11-15 13:30:00
2024-11-15 14:00:00
2024-11-15 14:30:00
2024-11-15 15:00:00
2024-11-15 15:30:00
2024-11-15 16:00:00
2024-11-15 16:30:00
2024-11-15 17:00:00
2024-11-15 17:30:00
2024-11-15 18:00:00
2024-11-15 18:30:00
2024-11-15 19:00:00
2024-11-15 19:30:00
2024-11-15 20:00:00
2024-11-15 20:30:00
2024-11-15 21:00:00
2024-11-15 21:30:00
2024-11-15 22:00:00
2024-11-15 22:30:00
2024-11-15 23:00:00
2024-11-15 23:30:00
2024-11-16 00:00:00
2024-11-16 00:30:00


In [17]:
values_list

[{'date': '2024-11-15 00:00:00', 'precipitation': 0},
 {'date': '2024-11-15 00:30:00', 'precipitation': 0},
 {'date': '2024-11-15 01:00:00', 'precipitation': 0},
 {'date': '2024-11-15 01:30:00', 'precipitation': 0},
 {'date': '2024-11-15 02:00:00', 'precipitation': 0},
 {'date': '2024-11-15 02:30:00', 'precipitation': 0},
 {'date': '2024-11-15 03:00:00', 'precipitation': 0},
 {'date': '2024-11-15 03:30:00', 'precipitation': 0},
 {'date': '2024-11-15 04:00:00', 'precipitation': 0},
 {'date': '2024-11-15 04:30:00', 'precipitation': 0},
 {'date': '2024-11-15 05:00:00', 'precipitation': 0.02499999850988388},
 {'date': '2024-11-15 05:30:00', 'precipitation': 0.03999999910593033},
 {'date': '2024-11-15 06:00:00', 'precipitation': 0.08999999612569809},
 {'date': '2024-11-15 06:30:00', 'precipitation': 0},
 {'date': '2024-11-15 07:00:00', 'precipitation': 0.03500000014901161},
 {'date': '2024-11-15 07:30:00', 'precipitation': 0},
 {'date': '2024-11-15 08:00:00', 'precipitation': 0.024999998509